# CFT – Questionnaire Analysis

## Setup and Helper Functions

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp

from cft_analysis.datasets import CftDatasetProcessed

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 4

palette

## Import Data

In [ ]:
# get path to analysis results
base_path = Path("../../data")

In [ ]:
results_path = base_path.joinpath("../results")
stats_path = results_path.joinpath("statistics")
tex_path = stats_path.joinpath("tex_tables")
bp.utils.file_handling.mkdirs([results_path, stats_path, tex_path])

paper_path = Path("../paper_path.json")
paper_tex_path = None
if paper_path.exists():
    paper_path = Path(json.load(paper_path.open(encoding="utf-8"))["paper_path"])
    paper_tex_path = paper_path.joinpath("tab")
    bp.utils.file_handling.mkdirs([paper_tex_path])

In [ ]:
dataset = CftDatasetProcessed(base_path, exclude_subjects=True)
dataset

In [ ]:
quest_data = dataset.questionnaire
quest_data.head()

## Statistical Analysis

### PSS

#### Prepare Data

In [ ]:
data_analysis = quest_data[["PSS_Total"]]
data_analysis.head()

#### Statistical Analysis

In [ ]:
steps = [
    ("prep", "normality"), 
    ("test", "pairwise_ttests")
]
params = {
    "dv": "PSS_Total", 
    "between": "condition"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(data_analysis)
stats.export_statistics(stats_path.joinpath("stats_questionnaire_pss.xlsx"))
stats.display_results()

### MDBF

#### Prepare Data

In [ ]:
# Drop the MDBF Total score, use only the subscales
data_analysis = quest_data.filter(like="MDBF").drop(columns=["MDBF_pre", "MDBF_post"])

# Convert to long-format and reindex data
data_analysis = bp.utils.dataframe_handling.wide_to_long(data_analysis, "MDBF", levels=["subscale", "time"])
data_analysis = data_analysis.reindex(["pre", "post"], level="time")
data_analysis.head()

#### Descriptive Analysis

Compute MIST-induced mood worsening in percent

In [ ]:
mdbf_diff = data_analysis.groupby(["subject", "subscale"]).apply(lambda df: (df.diff() / df.xs("pre", level="time")))
mdbf_diff = mdbf_diff.dropna().droplevel("time")
mdbf_diff = mdbf_diff * 100
mdbf_diff_percent = mdbf_diff.groupby(["condition", "subscale"]).agg(["mean", "std"])

In [ ]:
mdbf_diff_percent.to_csv(results_path.joinpath("questionnaire_mdbf_decrease.csv"))
mdbf_diff_percent.round(2)

#### Statistics

**Population**: Both conditions

**Analysis**: Check whether MIST caused a significant mood worsening, indicated by MBDF
* Procedure: Paired t-tests between MDBF subscales before and after MIST
* Expected Result: Significant mood worsening as response to the MIST

**Findings**: 
* The MIST causes an effective activation of the HPA axis, indicated by a significant increase in cortisol after the MIST for the Control condition

In [ ]:
steps = [
    ("prep", "normality"), 
    ("test", "pairwise_ttests")
]
params = {
    "dv": "MDBF", 
    "within": "time",
    "subject": "subject",
    "groupby": ["condition", "subscale"],
    "multicomp": {"levels": "subscale"}
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(data_analysis)
stats.export_statistics(stats_path.joinpath("stats_questionnaire_mdbf_pre_post.xlsx"))
stats.display_results()

display(Markdown("#### Latex Output"))
print(stats.stats_to_latex("pairwise_ttests", ("Control")))

#### Interaction *Condition x Time*

**Population**: Both conditions

**Analysis**: Check whether the Cold Face Test has an influence on mood worsening, indicated by MDBF
* Procedure: Mixed-ANOVA to determine interaction effect between Condition and Time for each MDBF subscale
* Expected Result: Significant interaction effect of CFT on MDBF scores

**Findings**: 
* No significant interaction effect

In [ ]:
steps = [
    ("prep", "normality"),
    ("prep", "equal_var"),
    ("test", "mixed_anova"),
    ("posthoc", "pairwise_ttests")
]
params = {
    "dv": "MDBF", 
    "between": "condition",
    "within": "time",
    "subject": "subject",
    "groupby": "subscale"
}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(data_analysis)
stats.export_statistics(stats_path.joinpath("stats_questionnaire_mdbf_interaction.xlsx"))
stats.display_results()